## Clean CSV code like a basic clean

In [21]:
import pandas as pd

# Load the CSV into a DataFrame
file_path = 'dbird_data_Georgia_Tech_final(1).csv' 
dfoG = pd.read_csv(file_path)

# Check for duplicate rows
duplicate_rows = dfoG[dfoG.duplicated()]
print(f"Number of duplicate rows: {len(duplicate_rows)}")

# Drop the columns 'notes' and 'image'
dfoG = dfoG.drop(columns=['notes', 'image', 'created_at_utc','updated_at_utc', 'location_accuracy_meters', 'mobile_device_gps_source'])


Number of duplicate rows: 0


In [ ]:
# Convert 'observed_at_utc' column to datetime
dfoG['observed_at_utc'] = pd.to_datetime(dfoG['observed_at_utc'], format='mixed', errors='coerce')

# Format the datetime to only include the date (YYYY-MM-DD)
dfoG['observed_at_utc'] = dfoG['observed_at_utc'].dt.date

In [23]:
# Save the updated DataFrame back to a CSV if needed
dfoG.to_csv('updated_dbird.csv', index=False)


## Use the new cleaned CSV data to calculate everything

In [24]:
import pandas as pd

file_path = 'updated_dbird.csv'
df = pd.read_csv('updated_dbird.csv')

# Define latitude and longitude bounds
top_latitude = 33.77592
bottom_latitude = 33.77495
left_longitude = -84.40163
right_longitude = -84.40106



In [25]:
# Convert 'observed_at_utc' column to datetime
df['observed_at_utc'] = pd.to_datetime(df['observed_at_utc'], format='mixed', errors='coerce')

# Format the datetime to only include the date (YYYY-MM-DD)
df['observed_at_utc'] = df['observed_at_utc'].dt.date

In [26]:
# Filter rows within the latitude and longitude range
df_2024_within_bounds = df[
    (df['latitude'] <= top_latitude) &
    (df['latitude'] >= bottom_latitude) &
    (df['longitude'] >= left_longitude) &
    (df['longitude'] <= right_longitude)
]
# Rows in 2024 but outside bounds
df_2024_outside_bounds = df.drop(df_2024_within_bounds.index)

In [27]:
df_2024_within_bounds.to_csv('dbird_2024_within_bounds.csv', index=False)
df_2024_outside_bounds.to_csv('dbird_2024_outside_bounds.csv', index=False)

print("Files saved successfully!")

Files saved successfully!


In [28]:
df_2024_outside_bounds.to_csv('dbird_2024_outside_bounds.csv', index=False)

print("Files saved successfully!")

Files saved successfully!


## Calcualting the Mean and std of dbird_2024_outside_bounds (df1) (data for Q1)

All the data saved from this run is in the yearly_data where it's split off by year

In [11]:
import pandas as pd
import os



# File path to the input dataset
file_path = "dbird_2024_outside_bounds.csv"

# Load the dataset
df1 = pd.read_csv(file_path)

# Ensure the 'observed_at_utc' column is in datetime format
df1['observed_at_utc'] = pd.to_datetime(df1['observed_at_utc'], errors='coerce')

# Extract the year from the 'observed_at_utc' column
df1['year'] = df1['observed_at_utc'].dt.year

# Folder to save the yearly CSV files
output_folder = "yearly_data"

# Create the folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Split the data into yearly CSV files (2017-2024)
for year in range(2017, 2025):
    year_df = df1[df1['year'] == year]  # Filter for the current year
    output_file = os.path.join(output_folder, f'dbird_data_{year}.csv')  # Create file path in the folder
    year_df.to_csv(output_file, index=False)  # Save to a CSV file
print("Success: Saved data for 2017-2024.")

# Initialize a list to store the sizes of each year's dataset
sizes = []

# Calculate the size (number of rows) for each CSV file
for year in range(2017, 2025):
    filepath = os.path.join(output_folder, f'dbird_data_{year}.csv')  # Update path to include folder
    try:
        tmeancalc = pd.read_csv(filepath)
        sizes.append(len(tmeancalc))  # Add the number of rows to the sizes list
    except FileNotFoundError:
        print(f"File {filepath} not found. Skipping...")

# Calculate the mean of the sizes
mean_size = sum(sizes) / len(sizes) 
mean_exsize = sum(sizes[4:]) / len(sizes)

# Calculate the standard deviation of the sizes
std_deviation = pd.Series(sizes).std()
# Calculate the standard deviation for the list excluding the first 4 elements, if applicable
if len(sizes) > 4:
    outliers_ex = pd.Series(sizes[4:]).std()
else:
    outliers_ex = 0

# Output the results
print(f"Number of files processed: {len(sizes)}")
print("")
print(f"Mean size of datasets: {mean_size}")
print(f"Mean excluding outliers {mean_exsize}")
print("")
print(f"Standard deviation of dataset sizes: {std_deviation}")
print(f"Standard Deviation excluding outliers: {outliers_ex}")
print("")
print(f"bird data within bounds mean: {len(pd.read_csv('dbird_2024_within_bounds.csv'))}")
print("bird data within bounds Standard deviation: 0")

Success: Saved data for 2017-2024.
Number of files processed: 8

Mean size of datasets: 61.25
Mean excluding outliers 60.875

Standard deviation of dataset sizes: 78.75413821326651
Standard Deviation excluding outliers: 68.62154666478064

bird data within bounds mean: 11
bird data within bounds Standard deviation: 0


## Calculating the bird Species (df2) (Extra)

In [30]:
import pandas as pd

file_path = "updated_dbird.csv"

df2 = pd.read_csv(file_path)

cleaned_df = df2[(df2["common_name"]!='unknown' )]

bird_counts = cleaned_df['common_name'].value_counts()

print(bird_counts)

common_name
Ovenbird                   44
Tennessee Warbler          28
Gray Catbird               22
American Robin             21
Swainson's Thrush          19
                           ..
White-breasted Nuthatch     1
Dark-eyed Junco             1
Carolina Chickadee          1
Red-shouldered Hawk         1
Hooded Warbler              1
Name: count, Length: 70, dtype: int64


# Calculate by Year which half has more collisions (data for Q3)

In [25]:
import pandas as pd
import os


# Folder to save the yearly CSV files
output_folder = "yearly_data"
Half1_folder = "yearly_data_spring"
Half2_folder = "yearly_data_Fall"


# Create the folder if it doesn't exist
os.makedirs(Half1_folder, exist_ok=True)
os.makedirs(Half2_folder, exist_ok=True)

for year in range(2017, 2025):
    filepath = os.path.join(output_folder, f'dbird_data_{year}.csv')  # Path to the yearly file
    
    try:
        # Read the file
        current_file = pd.read_csv(filepath)

        # Ensure the observed_at_utc is in datetime format
        current_file['observed_at_utc'] = pd.to_datetime(current_file['observed_at_utc'])

        # Filter for months 1-6 (Spring)
        first_half = current_file[current_file['observed_at_utc'].dt.month.isin(range(1, 7))]
        spring_output_file = os.path.join(Half1_folder, f'dbird_data_{year}_spring.csv')
        first_half.to_csv(spring_output_file, index=False)  # Save spring data to CSV

        # Filter for months 7-12 (Fall)
        second_half = current_file[current_file['observed_at_utc'].dt.month.isin(range(7, 13))]
        fall_output_file = os.path.join(Half2_folder, f'dbird_data_{year}_fall.csv')
        second_half.to_csv(fall_output_file, index=False)  # Save fall data to CSV


    except FileNotFoundError:
        print(f"File for year {year} not found at {filepath}. Skipping...")
    except Exception as e:
        print(f"An error occurred while processing year {year}: {e}")



In [24]:
# Folders to save the yearly CSV files
output_folder = "yearly_data"
Half1_folder = "yearly_data_spring"
Half2_folder = "yearly_data_fall"

# Initialize lists to store the sizes of each half's datasets
spring_sizes = []
fall_sizes = []

# Process each year's data
for year in range(2017, 2025):
    filepath = os.path.join(output_folder, f'dbird_data_{year}.csv')  # Path to the yearly file
    
    try:
        # Read the file
        current_file = pd.read_csv(filepath)

        # Ensure the observed_at_utc is in datetime format
        current_file['observed_at_utc'] = pd.to_datetime(current_file['observed_at_utc'])

        # Filter for months 1-6 (Spring)
        spring_data = current_file[current_file['observed_at_utc'].dt.month.isin(range(1, 7))]
        spring_sizes.append(len(spring_data))  # Store size of the spring dataset

        # Filter for months 7-12 (Fall)
        fall_data = current_file[current_file['observed_at_utc'].dt.month.isin(range(7, 13))]
        fall_sizes.append(len(fall_data))  # Store size of the fall dataset

    except FileNotFoundError:
        print(f"File for year {year} not found at {filepath}. Skipping...")
    except Exception as e:
        print(f"An error occurred while processing year {year}: {e}")

# Calculate statistics for Spring (Months 1-6)
spring_mean = sum(spring_sizes) / len(spring_sizes) 
spring_median = pd.Series(spring_sizes).median() 

# Calculate statistics for Fall (Months 7-12)
fall_mean = sum(fall_sizes) / len(fall_sizes) 
fall_median = pd.Series(fall_sizes).median() 

if len(sizes) > 4:
    Spring_outliers_ex = pd.Series(spring_sizes[4:]).std()
else:
    Spring_outliers_ex = 0
    
if len(sizes) > 4:
    Fall_outliers_ex = pd.Series(fall_sizes[4:]).std()
else:
    Fall_outliers_ex = 0


SPmean_exsize = sum(spring_sizes[4:]) / len(spring_sizes)
FALLmean_exsize = sum(fall_sizes[4:]) / len(fall_sizes)
# Output results
print("Spring (Months 1-6):")
print(f"Sizes: {spring_sizes}")
print(f"Mean: {spring_mean}")
print(f"Mean cleaned: {SPmean_exsize}")
print(f"Median: {spring_median}")
print(f"Standard Deviation: {pd.Series(spring_sizes).std()}")
print(f"Standard Deviation: {Spring_outliers_ex} (cleaned)")
print("")

print("Fall (Months 7-12):")
print(f"Sizes: {fall_sizes}")
print(f"Mean: {fall_mean}")
print(f"Mean cleaned: {FALLmean_exsize}")
print(f"Median: {fall_median}")
print(f"Standard Deviation: {pd.Series(fall_sizes).std()}")
print(f"Standard Deviation: {Fall_outliers_ex} (cleaned)")

Spring (Months 1-6):
Sizes: [0, 0, 0, 0, 1, 41, 52, 39]
Mean: 16.625
Mean cleaned: 16.625
Median: 0.5
Standard Deviation: 22.977862016670368
Standard Deviation: 22.246722605064026 (cleaned)

Fall (Months 7-12):
Sizes: [3, 0, 0, 0, 22, 139, 99, 94]
Mean: 44.625
Mean cleaned: 44.25
Median: 12.5
Standard Deviation: 56.7197055704629
Standard Deviation: 48.692915295759406 (cleaned)


## Data fro Q4